In [74]:
import os
import rasterio
import rioxarray
import numpy as np
import laspy as laspy 
#laspy is a good point cloud tool
import open3d as o3d


# Plotting setup
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'
#plt.rcParams.update({'font.size': 16}) # make matplotlib font sizes bigger

In [90]:
!cat learn-pdal-python_batch.json

{
    "pipeline": [
        {
           "type":"readers.las"
        },
        {
            "type":"filters.dem",
            "raster":"tandem.tif",
            "limits":"Z[20:35]"
        },
        {
            "type":"filters.outlier",
            "method":"statistical",
            "mean_k":12,
            "multiplier":2.2
        },
        {
          "type":"filters.range",
          "limits":"classification![7:7]"
        },
        {
            "type":"filters.smrf"
        },
        {
            "type":"filters.range",
            "limits":"Classification[2:2]"
        },
        {
           "type":"writers.las"
        },
        {
            "type":"writers.gdal",
            "resolution":1.0,
            "output_type":"idw"
        }
    ]
}


In [47]:
#get a list of files to push to pipeline
basepath = ('/Users/rdcrlsro/Documents/lidar_play/Bogus/test/')
def get_file_list(directory: str, file_types: str ='*.laz') -> list:
    return [f for f in Path(directory).glob(file_types) if f.is_file()]

files = get_file_list(basepath)
print(files)

[PosixPath('/Users/rdcrlsro/Documents/lidar_play/Bogus/test/2021-02-24-Bogus - VQ-580 - 210224_213433_VQ-580 - originalpoints.laz'), PosixPath('/Users/rdcrlsro/Documents/lidar_play/Bogus/test/2021-02-24-Bogus - VQ-580 - 210224_213528_VQ-580 - originalpoints.laz')]


In [91]:
#Here is the crux, I/O of a dir of files.
#!pdal pipeline learn-pdal-python.json --verbose 5 \

!ls /Users/rdcrlsro/Documents/lidar_play/Bogus/test/*.laz | \
parallel -I{} pdal pipeline ./learn-pdal-python_batch.json \
--readers.las.filename={} \
--writers.las.filename=/Users/rdcrlsro/Documents/lidar_play/Bogus/test/out/PC/clean{/.}.las \
--writers.gdal.filename=/Users/rdcrlsro/Documents/lidar_play/Bogus/test/out/Raster/clean{/.}.tif \
gdalbuildvrt  /Users/rdcrlsro/Documents/lidar_play/Bogus/test/out/Raster/dtm3.vrt /Users/rdcrlsro/Documents/lidar_play/Bogus/test/out/Raster*.tif

In [92]:
#combine the tif output
!ls /Users/rdcrlsro/Documents/lidar_play/Bogus/test/out/Raster/*.tif | \
gdalbuildvrt  /Users/rdcrlsro/Documents/lidar_play/Bogus/test/out/Raster/dtm3.vrt /Users/rdcrlsro/Documents/lidar_play/Bogus/test/out/Raster/*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


In [6]:
PC= laspy.read('/Users/rdcrlsro/Documents/lidar_play/MessyBogus-clean.las')
print('Points from Header:', PC.header.point_count)
print(PC)
print(np.unique(PC.classification))

list(PC.point_format.dimension_names)

Points from Header: 81526712
<LasData(1.2, point fmt: <PointFormat(3, 0 bytes of extra dims)>, 81526712 points, 3 vlrs)>
[2 4]


['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time',
 'red',
 'green',
 'blue']

In [7]:
points = np.vstack((PC.X, PC.Y, PC.Z)).transpose()

cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(points)
#pcd.colors = o3d.utility.Vector3dVector(colors/65535)
#pcd.normals = o3d.utility.Vector3dVector(normals)
o3d.visualization.draw_geometries([cloud])
